In [31]:
infile_pos='score-positive.4mer.features.csv'
infile_nez='score-negative.4mer.features.csv'
infile_zero='score-zero.4mer.features.csv'
import numpy as np
import pandas as pd
rawdata = pd.read_csv(infile_pos,header=0)
rawdata.shape
type(rawdata)

pandas.core.frame.DataFrame

In [32]:
randomdata.head()

,-1,ENSG00000259891,71,8,9,16,16.1,5,0,1,...,2.14,9.21,20,11.13,8.15,8.16,11.14,3.20,19.1,11.15
202,-1,ENSG00000275441,4896,2145,2659,3019,2222,1593,446,1692,...,468,2887,1839,1720,2221,2186,2483,2834,2785,5328
330,-1,ENSG00000271918,7846,3503,4499,4827,3627,2657,730,2900,...,765,4807,3039,2843,3645,3504,4020,4692,4564,8459
24,-1,ENSG00000267546,933,369,456,554,362,247,68,306,...,62,474,330,252,300,384,445,499,452,1051
992,0,ENSG00000255121,7011,3012,3865,4763,3170,2064,548,2581,...,469,4159,2749,2302,2844,3072,3927,4023,4050,7590
664,1,ENSG00000278231,2064,908,1162,1352,981,564,157,738,...,128,1081,740,627,804,841,1011,1023,1039,2004


pandas.core.frame.DataFrame

In [20]:
randomdata = rawdata.sample(frac=1)
type(randomdata)

pandas.core.frame.DataFrame

In [21]:
randomdata.shape

(1151, 258)

,-1,ENSG00000259891,71,8,9,16,16.1,5,0,1,...,2.14,9.21,20,11.13,8.15,8.16,11.14,3.20,19.1,11.15
202,-1,ENSG00000275441,4896,2145,2659,3019,2222,1593,446,1692,...,468,2887,1839,1720,2221,2186,2483,2834,2785,5328
330,-1,ENSG00000271918,7846,3503,4499,4827,3627,2657,730,2900,...,765,4807,3039,2843,3645,3504,4020,4692,4564,8459
24,-1,ENSG00000267546,933,369,456,554,362,247,68,306,...,62,474,330,252,300,384,445,499,452,1051
992,0,ENSG00000255121,7011,3012,3865,4763,3170,2064,548,2581,...,469,4159,2749,2302,2844,3072,3927,4023,4050,7590
664,1,ENSG00000278231,2064,908,1162,1352,981,564,157,738,...,128,1081,740,627,804,841,1011,1023,1039,2004
